## Drive connection

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installations and Imports

In [2]:
!pip install roboflow
!pip install -U ultralytics wandb

# Enable W&B logging for Ultralytics
!yolo settings wandb=True

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 123.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.3 MB/s eta 0:00:00
   

In [3]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tropdesucre (tropdesucre-t-l-com-sudparis) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
import shutil
from roboflow import Roboflow
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
import shutil
import random
from pathlib import Path
import yaml
import wandb
import torch
import pandas as pd
from wandb.integration.ultralytics import add_wandb_callback

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


## Dataset

In [6]:
rf = Roboflow(api_key="api_key")
project = rf.workspace("thesis-2zk7q").project("hq-hole-detection-moving-objects")
version = project.version(9)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to HQ-+-hole-detection,-moving-objects-9 in yolov11:: 100%|██████████| 11164/11164 [00:01<00:00, 6488.11it/s]


### Modifying dataset split ratio


In [ ]:
# Set seed for reproducibility
random.seed(42)

#original dataset location
original_dataset_path = dataset.location

# Output directory
resplit_dataset_path = os.path.join("/content/drive/MyDrive/roboflow_datasets", "resplit")
os.makedirs(resplit_dataset_path, exist_ok=True)

#new folders for train/val/test
splits = ["train", "valid", "test"]
for split in splits:
    os.makedirs(os.path.join(resplit_dataset_path, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(resplit_dataset_path, split, "labels"), exist_ok=True)

# Gather all images and labels from original splits
def collect_images_labels(folder):
    image_dir = os.path.join(original_dataset_path, folder, "images")
    label_dir = os.path.join(original_dataset_path, folder, "labels")
    image_files = sorted([f for f in Path(image_dir).glob("*.jpg")])
    paired = [(img, Path(label_dir) / (img.stem + ".txt")) for img in image_files]
    return paired

all_data = []
for split in ["train", "valid", "test"]:
    all_data.extend(collect_images_labels(split))

print(f"Total examples found: {len(all_data)}")

# Shuffle and split 70/15/15
random.shuffle(all_data)
n_total = len(all_data)
n_train = int(0.70 * n_total)
n_valid = int(0.15 * n_total)
n_test  = n_total - n_train - n_valid

train_data = all_data[:n_train]
valid_data = all_data[n_train:n_train + n_valid]
test_data  = all_data[n_train + n_valid:]

print(f"Split: {len(train_data)} train / {len(valid_data)} valid / {len(test_data)} test")

# Move files to new structure
def move_pairs(pairs, dest_split):
    for img, label in pairs:
        shutil.copy(img, os.path.join(resplit_dataset_path, dest_split, "images", img.name))
        shutil.copy(label, os.path.join(resplit_dataset_path, dest_split, "labels", label.name))

move_pairs(train_data, "train")
move_pairs(valid_data, "valid")
move_pairs(test_data, "test")

# Get class names from original YAML
original_yaml_path = os.path.join(original_dataset_path, "data.yaml")
with open(original_yaml_path, "r") as f:
    original_yaml = yaml.safe_load(f)
class_names = original_yaml["names"]
nc = original_yaml["nc"]

#new data.yaml
resplit_yaml = {
    "path": resplit_dataset_path,
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "nc": nc,
    "names": class_names,
}

resplit_yaml_path = os.path.join(resplit_dataset_path, "data.yaml")
with open(resplit_yaml_path, "w") as f:
    yaml.dump(resplit_yaml, f)

print(f" New data.yaml saved at: {resplit_yaml_path}")


Total examples found: 5576
Split: 3903 train / 836 valid / 837 test
✅ New data.yaml saved at: /content/drive/MyDrive/roboflow_datasets/resplit/data.yaml


## Hyperparameter optimization and training

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

#random search sweep configuration
sweep_config = {
    "method": "random",
    "metric": {
        "name": "metrics/mAP_0.5",
        "goal": "maximize"
    },
    "parameters": {
        "epochs": {"values": [10]},
        "batch": {"values": [16, 32]},
        "lr0": {"values": [1e-3, 1e-4]},
        "weight_decay": {"values": [1e-4, 1e-3]},
        "optimizer": {"values": ["SGD", "Adam"]}
    }
}

data_yaml = "/content/drive/MyDrive/roboflow_datasets/resplit/data.yaml"

#training function used by the WandB agent
def train():
    run = wandb.init(
        project="yolo-seg-worker-randomsearch",
        job_type='training'
    )
    config = run.config
    run_name = f"e{config.epochs}_b{config.batch}_lr{config.lr0}_wd{config.weight_decay}_{config.optimizer}"
    wandb.run.name = run_name

    # Load the model and attach WandB callback
    model = YOLO("/content/yolo11n-seg.pt")
    add_wandb_callback(model, enable_model_checkpointing=True)

    # Train the model using sweep parameters
    model.train(
        data=data_yaml,
        epochs=config.epochs,
        batch=config.batch,
        lr0=config.lr0,
        weight_decay=config.weight_decay,
        optimizer=config.optimizer,
        project="yolo-seg-worker-randomsearch",
        name=run_name,
        device=0 if torch.cuda.is_available() else "cpu",
        val=True
    )

    run.finish()


#Launch sweep
sweep_id = wandb.sweep(sweep_config, project="yolo-seg-worker-randomsearch")
wandb.agent(sweep_id, function=train, count=6)


Using device: cuda
Create sweep with ID: tn9hj9dw
Sweep URL: https://wandb.ai/tropdesucre-t-l-com-sudparis/yolo-seg-worker-randomsearch/sweeps/tn9hj9dw


wandb: Agent Starting Run: fsb5ppm5 with config:
wandb: 	batch: 32
wandb: 	epochs: 10
wandb: 	lr0: 0.0001
wandb: 	optimizer: Adam
wandb: 	weight_decay: 0.0001
wandb: Currently logged in as: tropdesucre (tropdesucre-t-l-com-sudparis) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


100%|██████████| 5.90M/5.90M [00:00<00:00, 273MB/s]


Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/roboflow_datasets/resplit/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=e10_b32_lr0.0001_wd0.0001_Adam, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=Tr

100%|██████████| 755k/755k [00:00<00:00, 86.1MB/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:08<00:00,  1.63it/s]


                   all        836       4122      0.809      0.538       0.62      0.423      0.787      0.521       0.59      0.338
               machine        579       1674      0.785      0.517      0.591      0.434      0.764        0.5      0.558      0.322
                worker        647       2448      0.833      0.558      0.649      0.412       0.81      0.541      0.622      0.353


Generating Visualizations for batch-1/1:   0%|          | 0/64 [00:00<?, ?it/s]

Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to yolo-seg-worker-randomsearch/e10_b32_lr0.0001_wd0.0001_Adam2


lr/pg0,█▅▁▁▁▁▁▁▁▁
lr/pg1,▃▆█▇▆▅▄▃▂▁
lr/pg2,▃▆█▇▆▅▄▃▂▁
metrics/mAP50(B),▁▆▇▇▇█████
metrics/mAP50(M),▁▆▇▇██████
metrics/mAP50-95(B),▁▅▆▇▇▇████
metrics/mAP50-95(M),▁▅▇▇▇█████
metrics/precision(B),▁▇▇██▇████
metrics/precision(M),▁▇████████
metrics/recall(B),▃▁▄▅▅▇▆▇██
metrics/recall(M),▁▆▇▇▇█████


### Training with best hyperparameters

In [7]:
import wandb
import os
from ultralytics import YOLO

# ==== CONFIG ====
EPOCHS = 100
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.0001
OPTIMIZER = 'Adam'
MODEL_NAME = 'yolo11n-seg.pt'
PROJECT_NAME = 'yolov11n-segmentation'
RUN_NAME = f"e{EPOCHS}_b{BATCH_SIZE}_lr{LEARNING_RATE}_wd{WEIGHT_DECAY}_{OPTIMIZER}"


wandb.init(project=PROJECT_NAME, name=RUN_NAME, config={
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "weight_decay": WEIGHT_DECAY,
    "optimizer": OPTIMIZER,
    "model": MODEL_NAME
})

model = YOLO(MODEL_NAME)


results = model.train(
    data="/content/drive/MyDrive/roboflow_datasets/resplit/data.yaml",
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=640,
    optimizer=OPTIMIZER,
    lr0=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    project=PROJECT_NAME,
    name=RUN_NAME,
    save=True,
    val=True,
    verbose=True,
    patience=10
)

wandb.finish()


Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/roboflow_datasets/resplit/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=e100_b32_lr0.0001_wd0.0001_Adam, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, 

100%|██████████| 755k/755k [00:00<00:00, 117MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n-seg summary: 203 layers, 2,842,998 parameters, 2,842,982 gradients, 10.4 GFLOPs

Transferred 510/561 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 350MB/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  14%|█▍        | 2/14 [00:01<00:11,  1.04it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:08<00:00,  1.66it/s]


                   all        836       4122      0.868      0.614      0.697      0.497      0.848      0.593      0.668        0.4
               machine        579       1674      0.865      0.616      0.689      0.533      0.844      0.595       0.66      0.407
                worker        647       2448       0.87      0.612      0.705      0.462      0.852       0.59      0.676      0.393
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to yolov11n-segmentation/e100_b32_lr0.0001_wd0.0001_Adam


lr/pg0,████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg1,▃████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
lr/pg2,█████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▁
metrics/mAP50(B),▁▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████████████████
metrics/mAP50(M),▁▅▇▇▇▇▇▇▇▇██████████████████████████████
metrics/mAP50-95(B),▁▄▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
metrics/mAP50-95(M),▁▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████████████████
metrics/precision(B),▁▂▅▅▆▆▆▆▆▆▇▆▆▆▆▇▇▇▇▇▇▇█▇▇▇▇▇▇▇████▇█████
metrics/precision(M),▁▃▄▅▃▅▅▇▆▇▆▆▅▇▆▇▇▆▇▇▇▇▇▆▆▇█▇▇▇███████▇▇█
metrics/recall(B),▁▄▄▄▄▆▅▆▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇█▇▇█████████████
metrics/recall(M),▁▁▂▂▂▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▆▆▇▇▇▇▇█▇██▇▇████


## Evaluation on test set

In [10]:
# Load trained model weights
trained_model = YOLO("/content/yolov11n-segmentation/e100_b32_lr0.0001_wd0.0001_Adam/weights/best.pt")

# Run evaluation on the test set
test_results = trained_model.val(
    data="/content/drive/MyDrive/roboflow_datasets/resplit/data.yaml",
    split="test",
    imgsz=640,
    batch=32,
    verbose=True
)

print(test_results)


Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11n-seg summary (fused): 113 layers, 2,834,958 parameters, 0 gradients, 10.2 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 0.1±0.0 MB/s, size: 49.0 KB)


val: Scanning /content/drive/MyDrive/roboflow_datasets/resplit/test/labels... 837 images, 1 backgrounds, 0 corrupt: 100%|██████████| 837/837 [01:25<00:00,  9.82it/s]


val: New cache created: /content/drive/MyDrive/roboflow_datasets/resplit/test/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/27 [00:00<?, ?it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   7%|▋         | 2/27 [00:02<00:31,  1.26s/it]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:10<00:00,  2.59it/s]


                   all        837       3965      0.856      0.634      0.717      0.516      0.865      0.609       0.69      0.418
               machine        554       1458      0.858      0.651      0.721      0.561      0.872      0.634        0.7       0.44
                worker        647       2507      0.855      0.616      0.713      0.471      0.859      0.584      0.681      0.396
Speed: 0.6ms preprocess, 3.1ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/segment/val
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a626a12f790>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.0020